# Inspect markdown files - KNBS and CBK
In this notebook, the core objective is to inspect and ingest the text from the already processed markdown files for CBK and KNBS.

In [8]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import logging
import warnings
import os
import pandas as pd
warnings.filterwarnings("ignore")
from pathlib import Path

# Configure logging to see output in the notebook
logging.basicConfig(level=logging.INFO, stream=sys.stdout, force=True)

In [5]:
# Fix paths for src files
project_root = Path(os.getcwd()).parent
script_dir = project_root / "src"
if str(script_dir) not in sys.path:
    sys.path.append(str(script_dir))

In [6]:
from load.start_ollama import start_ollama_server
from load.ingest_md import ingest_markdown_reports

In [7]:
start_ollama_server()

✅ Ollama is already running.


True

In [4]:
# Define your paths
SCRATCH_DIR = os.environ.get("SCRATCH")
KNBS_MARKDOWN_DIR = os.path.join(SCRATCH_DIR, "mshauri-fedha/data/knbs/marker-output")
VECTOR_DB_PATH = "mshauri_fedha_chroma_db"
EMBEDDING_MODEL = "nomic-embed-text"

In [5]:
# Run the ingestion on KNBS text
ingest_markdown_reports(
    markdown_dir=KNBS_MARKDOWN_DIR,
    vector_db_path=VECTOR_DB_PATH,
    model=EMBEDDING_MODEL
)

📄 Scanning for Markdown Reports in /capstor/scratch/cscs/tligawa/mshauri-fedha/data/knbs/marker-output...


100%|██████████| 574/574 [00:00<00:00, 4626.46it/s]


   Loaded 574 report files.
   ✂️  Split into 32717 chunks.
🧠 Appending to Vector Store...
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Ingesting Reports: 100%|██████████| 32717/32717 [1:09:38<00:00,  7.83chunk/s]


✅ Reports Added. Hybrid Knowledge Base is ready.


In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [10]:
CBK_MARKDOWN_DIR = os.path.join(SCRATCH_DIR, "mshauri-fedha/data/cbk/marker-output")

In [11]:
# Run the ingestion on CBK text
ingest_markdown_reports(
    markdown_dir=CBK_MARKDOWN_DIR,
    vector_db_path=VECTOR_DB_PATH,
    model=EMBEDDING_MODEL
)

📄 Scanning for Markdown Reports in /capstor/scratch/cscs/tligawa/mshauri-fedha/data/cbk/marker-output...


100%|██████████| 958/958 [00:11<00:00, 79.89it/s] 


   Loaded 958 report files.
   ✂️  Split into 4582 chunks.
🧠 Appending to Vector Store...


Ingesting Reports: 100%|██████████| 4582/4582 [10:21<00:00,  7.37chunk/s]


✅ Reports Added. Hybrid Knowledge Base is ready.


## Test the performance
Using similarity search, we test the performance of the embedding model used here.

In [15]:
# Connect
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url="http://127.0.0.1:25000")
vectorstore = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embeddings)

# Query for a known table
query = "Interest rates commercial banks 2020" 

results = vectorstore.similarity_search(query, k=5)

print(f"🔎 Checking for tables in: '{query}'\n")

for i, doc in enumerate(results):
    content = doc.page_content
    print("---------------------------------------------------")
    print(f"Source: {doc.metadata.get('source', 'N/A').split('/')[-1].split('.')[0].replace('-', '')}")
    # Print first 500 chars to see if headers align
    print(content[:1500] + "...\n")

🔎 Checking for tables in: 'Interest rates commercial banks 2020'

---------------------------------------------------
📰 Source: knbs_batch_42_193220_2020kenyafactsfigures
| Value of shares traded (KSh Bn)         | 209   | 147   | 172   | 176   | 154   |
| Equities market capitalization (KSh Bn) | 2,054 | 1,932 | 2,522 | 2,102 | 2,540 |
| NSE 20 Share Index (Base Jan 1966=100)  | 4,040 | 3,186 | 3,712 | 2,834 | 2,654 |

<sup>1</sup> W eig hted av erag e commercial bank interes t rates

# **Monetary and Financial Statistics**

**Table 21: Commercial Banks' Deposits, Loans and Advances, 2015 - 2019**

| 2015                                           | 2016        | 2017        | 2018        | 2019*       |             |
|------------------------------------------------|-------------|-------------|-------------|-------------|-------------|
| Commercial bank<br>s (KS<br>h million)         |             |             |             |             |             |
| Deposits<br>liabilities     